## Conectar al Drive para acceder a los datos

In [1]:

try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE = "/content/drive/MyDrive"
except Exception as e:
    print("No estás en Colab o ya está montado. Usa ruta local.")
    BASE = "."


Mounted at /content/drive


In [2]:
import os, glob
import pandas as pd
import unicodedata

# Carpeta donde están los Excel
CARPETA = os.path.join(BASE, "DATA_LIMPIEZA")

# Busca sólo .xlsx (si tuvieras .xls cambia el patrón o agrega otro glob)
rutas = sorted(glob.glob(os.path.join(CARPETA, "*.xlsx")))

print(f"Encontrados {len(rutas)} archivos en: {CARPETA}")
for r in rutas[:3]:
    print("•", os.path.basename(r))


Encontrados 23 archivos en: /content/drive/MyDrive/DATA_LIMPIEZA
• CuidadCapital.xlsx
• Elprogreso.xlsx
• Guatemala.xlsx


## Normalizar los nombre de archivos

In [3]:
def normaliza(texto: str) -> str:
    if texto is None:
        return ""
    # quita acentos
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(ch for ch in texto if not unicodedata.combining(ch))
    # limpieza básica
    texto = texto.strip()
    texto = texto.replace("\n", " ").replace("\r", " ")
    # espacios -> guiones bajos, minúsculas
    texto = "_".join(texto.split())
    return texto.lower()


## Mapea cada uno de ellos y muestra sus encabezados, Si hay una columna vacia la elimina

In [6]:
encabezados_por_archivo = []  # lista de dicts para DataFrame resumen

for ruta in rutas:
    nombre = os.path.basename(ruta)
    try:
        # Lee solo encabezados (nrows=0) de la PRIMERA hoja
        df0 = pd.read_excel(ruta, sheet_name=0, nrows=0, engine="openpyxl")

        # Detectar columnas Unnamed
        cols_original = list(map(str, df0.columns.tolist()))
        cols_unnamed = [c for c in cols_original if c.startswith("Unnamed")]

        if cols_unnamed:
            print(f"\n⚠ {nombre} contiene columnas vacías: {cols_unnamed}")
            # Eliminar columnas Unnamed
            df0 = df0.loc[:, ~df0.columns.str.contains('^Unnamed')]
            # Actualizar lista de columnas originales sin las vacías
            cols_original = list(map(str, df0.columns.tolist()))

        # Normalizar nombres
        cols_normal = [normaliza(c) for c in cols_original]

        print(f"\n===== {nombre} =====")
        print("Encabezados (limpios):")
        print(cols_original)

        encabezados_por_archivo.append({
            "archivo": nombre,
            "num_columnas": len(cols_original),
            "encabezados_original": cols_original,
            "encabezados_normalizado": cols_normal,
            "set_normalizado": tuple(sorted(set(cols_normal))),
        })

    except Exception as e:
        print(f"\n===== {nombre} =====")
        print("ERROR al leer:", e)



⚠ CuidadCapital.xlsx contiene columnas vacías: ['Unnamed: 1']

===== CuidadCapital.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Elprogreso.xlsx contiene columnas vacías: ['Unnamed: 1']

===== Elprogreso.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Guatemala.xlsx contiene columnas vacías: ['Unnamed: 1']

===== Guatemala.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Izabal.xlsx contiene columnas vac

## Compara si cada archivo tiene las misma columnas

In [7]:
res = pd.DataFrame(encabezados_por_archivo)

# Encuentra el "esquema modal" (el conjunto de columnas normalizadas más frecuente)
conteo_sets = res["set_normalizado"].value_counts()
esquema_modal = conteo_sets.index[0] if not conteo_sets.empty else tuple()
print("\n\n=== Resumen de esquemas ===")
print(conteo_sets)

# Marca si cada archivo coincide con el esquema modal
res["coincide_modal"] = res["set_normalizado"].apply(lambda s: s == esquema_modal)

# Para ver rápidamente quiénes NO coinciden
no_match = res[~res["coincide_modal"]].copy()

print("\nArchivos que NO coinciden con el esquema modal:")
if no_match.empty:
    print("✅ Todos coinciden con el mismo conjunto de encabezados (normalizados).")
else:
    for _, row in no_match.iterrows():
        base = set(esquema_modal)
        actual = set(row["set_normalizado"])
        faltantes = sorted(list(base - actual))
        extras = sorted(list(actual - base))
        print(f"\n• {row['archivo']}")
        print("  - Faltan (vs modal):", faltantes if faltantes else "—")
        print("  - Extras (vs modal):", extras if extras else "—")




=== Resumen de esquemas ===
set_normalizado
(area, codigo, departamental, departamento, direccion, director, distrito, establecimiento, jornada, modalidad, municipio, nivel, plan, sector, status, supervisor, telefono)    23
Name: count, dtype: int64

Archivos que NO coinciden con el esquema modal:
✅ Todos coinciden con el mismo conjunto de encabezados (normalizados).
